# Backend Chat Functionality Test

このnotebookはバックエンドのチャット機能だけを独立してテストするためのものです。
既存のsrc/services層からimportして動作確認を行います。

## 1. 環境設定とImport

In [1]:
import sys
import os
from pathlib import Path

# プロジェクトのルートディレクトリをパスに追加
project_root = Path().cwd().parent
sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")
print(f"Python path: {sys.path[:3]}...")

Project root: /Users/kazukikomura/Developer/research/multiagentahp
Python path: ['/Users/kazukikomura/Developer/research/multiagentahp', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python39.zip', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9']...


In [2]:
# 環境変数の読み込み
from dotenv import load_dotenv
load_dotenv(project_root / '.env')

# 必要なライブラリのimport
import json
from typing import Dict, Any

print("Environment loaded successfully")

Environment loaded successfully


## 2. 既存サービスのImport

In [3]:
try:
    # 既存のサービス層からimport
    from src.services.langgraph_service import LangGraphService
    from src.services.simple_llm import SimpleLLMResponder
    print("✅ Services imported successfully")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Please make sure you're running this from the notebooks directory")

✅ Services imported successfully


## 3. テスト用サンプルデータの準備

In [5]:
# サンプルセッションデータ
sample_session_data = {
    'student_info': {
        'name': '田中太郎',
        'student_id': 'S2024001',
        'detailed_scores': {
            '学業成績': 3.2,
            '研究能力': 2.8,
            'コミュニケーション': 3.5,
            'リーダーシップ': 2.3,
            '将来性': 3.0
        }
    },
    'decision_data': {
        'user_weights': {
            '学業成績': 0.3,
            '研究能力': 0.25,
            'コミュニケーション': 0.2,
            'リーダーシップ': 0.1,
            '将来性': 0.15
        },
        'bot_evaluators': {
            'bot1': {'decision': '不合格'},
            'bot2': {'decision': '不合格'},
            'bot3': {'decision': '合格'}
        }
    },
    'rule_summary': {
        'criteria': '5項目の加重平均による総合評価',
        'threshold_description': '各項目2.5点以上が基準',
        'evaluation_method': '重み配分に基づく加重平均と多数決'
    },
    'threshold': 2.5,
    'last_user_text': ''
}

# 初期セッション状態
initial_state = {
    'turn': 0,
    'stage': 0.0,
    'route': 'Rules',
    'appeal_made': False,
    'rules_shown': False,
    'questions': []
}

print("✅ Sample data prepared")
print(f"User weights: {sample_session_data['decision_data']['user_weights']}")
print(f"Student scores: {sample_session_data['student_info']['detailed_scores']}")

✅ Sample data prepared
User weights: {'学業成績': 0.3, '研究能力': 0.25, 'コミュニケーション': 0.2, 'リーダーシップ': 0.1, '将来性': 0.15}
Student scores: {'学業成績': 3.2, '研究能力': 2.8, 'コミュニケーション': 3.5, 'リーダーシップ': 2.3, '将来性': 3.0}


## 4. LangGraphService（メイン）のテスト

In [6]:
# LangGraphServiceの初期化
lg_service = LangGraphService(enable_logging=False, session_id="test_session_001")
print("✅ LangGraphService initialized")

# 現在の状態をコピー（各ターンで更新されるため）
current_state = initial_state.copy()

✅ LangGraphService initialized


/Users/kazukikomura/Developer/research/multiagentahp/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [8]:
# ターン1: 初回接触（ルール説明）
print("=== ターン1: 初回接触 ===")
result1 = lg_service.execute_turn(
    message="こんにちは。今回の評価について説明をお願いします。",
    decision="未定",
    state=current_state,
    session_data=sample_session_data
)

# 状態を更新
current_state = result1['state']

print(f"AI Response: {result1['response']}")
print(f"Action: {result1['action']}")
print(f"Turn: {result1['turn']}")
print(f"Current Stage: {current_state['stage']}")
print("-" * 50)

=== ターン1: 初回接触 ===
DEBUG: Generating stage1_prefs_ack with payload: {'user_weights': {'学業成績': 0.3, '研究能力': 0.25, 'コミュニケーション': 0.2, 'リーダーシップ': 0.1, '将来性': 0.15}, 'last_user_text': 'こんにちは。今回の評価について説明をお願いします。'}
DEBUG: Generated ack: あなたは評価についての説明を求めていますね。次に、評価の観点を整理していきましょう。
DEBUG: bot_evaluators: {'bot1': {'decision': '不合格'}, 'bot2': {'decision': '不合格'}, 'bot3': {'decision': '合格'}}, bot_summary: {'bot1': '不合格', 'bot2': '不合格', 'bot3': '合格'}
DEBUG: Generating stage2_present with payload: {'profile_facts': {'name': '田中太郎', 'student_id': 'S2024001', 'detailed_scores': {'学業成績': 3.2, '研究能力': 2.8, 'コミュニケーション': 3.5, 'リーダーシップ': 2.3, '将来性': 3.0}}, 'user_weights': {'学業成績': 0.3, '研究能力': 0.25, 'コミュニケーション': 0.2, 'リーダーシップ': 0.1, '将来性': 0.15}, 'minority': True, 'bot_evaluators': {'bot1': '不合格', 'bot2': '不合格', 'bot3': '合格'}, 'ratio': '2対1'}
DEBUG: Generated analysis: ### 田中太郎さんの評価結果

#### 評価概要
- **学業成績**: 3.2
- **研究能力**: 2.8
- **コミュニケーション**: 3.5
- **リーダーシップ**: 2.3
- **将来性**: 3.0

#### 重み付け
- **学業成績**: 30

KeyError: 'turn'

In [ ]:
# ターン2: 重視点の確認
print("=== ターン2: 重視点の確認 ===")
result2 = lg_service.execute_turn(
    message="学業成績と研究能力を特に重視しています。基礎的な学力がしっかりしている学生を評価したいと思います。",
    decision="未定",
    state=current_state,
    session_data=sample_session_data
)

current_state = result2['state']

print(f"AI Response: {result2['response']}")
print(f"Action: {result2['action']}")
print(f"Turn: {result2['turn']}")
print(f"Current Stage: {current_state['stage']}")
print("-" * 50)

In [ ]:
# ターン3: 質問をしてみる
print("=== ターン3: 質問 ===")
result3 = lg_service.execute_turn(
    message="リーダーシップの評価が低いのはなぜですか？この項目はどのように評価されているのでしょうか？",
    decision="未定",
    state=current_state,
    session_data=sample_session_data
)

current_state = result3['state']

print(f"AI Response: {result3['response']}")
print(f"Action: {result3['action']}")
print(f"Turn: {result3['turn']}")
print(f"Current Stage: {current_state['stage']}")
print("-" * 50)

In [ ]:
# ターン4: 異議申し立て
print("=== ターン4: 異議申し立て ===")
result4 = lg_service.execute_turn(
    message="この評価には納得できません。リーダーシップの評価をもう一度見直してください。",
    decision="合格",
    state=current_state,
    session_data=sample_session_data
)

current_state = result4['state']

print(f"AI Response: {result4['response']}")
print(f"Action: {result4['action']}")
print(f"Turn: {result4['turn']}")
print(f"Appeal Made: {current_state.get('appeal_made', False)}")
print(f"Current Stage: {current_state['stage']}")
print("-" * 50)

In [ ]:
# ターン5: 最終まとめ
print("=== ターン5: 最終まとめ ===")
result5 = lg_service.execute_turn(
    message="分かりました。ありがとうございました。",
    decision="合格",
    state=current_state,
    session_data=sample_session_data
)

current_state = result5['state']

print(f"AI Response: {result5['response']}")
print(f"Action: {result5['action']}")
print(f"Turn: {result5['turn']}")
print(f"Current Stage: {current_state['stage']}")
print("-" * 50)

## 5. SimpleLLMResponder（軽量版）のテスト

In [ ]:
# SimpleLLMResponderのテスト
print("=== SimpleLLMResponder Test ===")
try:
    simple_responder = SimpleLLMResponder()
    
    # テストメッセージ
    test_message = "この学生の評価について詳しく教えてください。特に研究能力の部分が気になります。"
    
    simple_response = simple_responder.generate(
        user_message=test_message,
        decision_data=sample_session_data['decision_data'],
        fallback_decision="合格"
    )
    
    print(f"User Message: {test_message}")
    print(f"Simple LLM Response: {simple_response}")
    
except Exception as e:
    print(f"❌ SimpleLLMResponder error: {e}")

## 6. 会話履歴の表示

In [ ]:
# 全会話履歴の表示
print("=== 会話履歴 ===")
conversation_history = [
    {"turn": 1, "user": "こんにちは。今回の評価について説明をお願いします。", "ai": result1['response']},
    {"turn": 2, "user": "学業成績と研究能力を特に重視しています。基礎的な学力がしっかりしている学生を評価したいと思います。", "ai": result2['response']},
    {"turn": 3, "user": "リーダーシップの評価が低いのはなぜですか？この項目はどのように評価されているのでしょうか？", "ai": result3['response']},
    {"turn": 4, "user": "この評価には納得できません。リーダーシップの評価をもう一度見直してください。", "ai": result4['response']},
    {"turn": 5, "user": "分かりました。ありがとうございました。", "ai": result5['response']}
]

for conv in conversation_history:
    print(f"\n【Turn {conv['turn']}】")
    print(f"User: {conv['user']}")
    print(f"AI: {conv['ai']}")
    print("-" * 40)

## 7. 満足度スコアの分析

In [ ]:
# 各ターンの満足度スコアを収集
satisfaction_data = [
    {"turn": 1, "scores": result1.get('satisfaction_scores', {})},
    {"turn": 2, "scores": result2.get('satisfaction_scores', {})},
    {"turn": 3, "scores": result3.get('satisfaction_scores', {})},
    {"turn": 4, "scores": result4.get('satisfaction_scores', {})},
    {"turn": 5, "scores": result5.get('satisfaction_scores', {})}
]

print("=== 満足度スコアの推移 ===")
for data in satisfaction_data:
    print(f"\nTurn {data['turn']}:")
    for dimension, score in data['scores'].items():
        print(f"  {dimension}: {score}")

# 最終的な平均スコア
if result5.get('satisfaction_scores'):
    final_scores = result5['satisfaction_scores']
    average_score = sum(final_scores.values()) / len(final_scores)
    print(f"\n最終平均満足度: {average_score:.2f}")

## 8. インタラクティブテスト（オプション）

In [ ]:
# インタラクティブなチャットテスト用の関数
def interactive_chat_test():
    """インタラクティブなチャットテスト"""
    print("=== Interactive Chat Test ===")
    print("Type 'quit' to exit")
    
    # 新しいセッションを開始
    interactive_service = LangGraphService(enable_logging=False, session_id="interactive_test")
    interactive_state = initial_state.copy()
    
    while True:
        user_input = input("\nUser: ").strip()
        if user_input.lower() == 'quit':
            print("Chat ended.")
            break
            
        if not user_input:
            continue
            
        try:
            result = interactive_service.execute_turn(
                message=user_input,
                decision="未定",
                state=interactive_state,
                session_data=sample_session_data
            )
            
            interactive_state = result['state']
            
            print(f"AI: {result['response']}")
            print(f"[Turn: {result['turn']}, Stage: {interactive_state['stage']}, Action: {result['action']}]")
            
        except Exception as e:
            print(f"Error: {e}")

# インタラクティブテストを実行する場合は下記をアンコメント
# interactive_chat_test()

## 9. まとめ

このnotebookでは以下をテストしました：

1. **LangGraphService**: メインのチャットサービス
   - ルール説明 → 重視点確認 → 観点整理 → 質問対応 → 異議申し立て → まとめ
   - 手続き的公正の各段階を適切に処理

2. **SimpleLLMResponder**: 軽量版チャットサービス
   - シンプルな質問応答機能

3. **満足度スコア**: 各段階での手続き的公正満足度の測定

4. **インタラクティブテスト**: リアルタイムでの動作確認

既存のservice層を活用して、チャット機能を独立してテストできる環境が完成しました。

In [9]:
# テスト実行
import sys
import os
from pathlib import Path

# プロジェクトルートをパスに追加
project_root = Path('/Users/kazukikomura/Developer/research/multiagentahp')
sys.path.insert(0, str(project_root))

print(f"✅ プロジェクトルート設定: {project_root}")

✅ プロジェクトルート設定: /Users/kazukikomura/Developer/research/multiagentahp
